# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](https://github.com/aicourse2025/lab-neural-networks/blob/master/your-code/tttboard.jpg?raw=1)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [6]:
import pandas as pd

ttt_data = pd.read_csv("./sample_data/tic-tac-toe.csv")

ttt_data.replace({'x': 1, 'o': -1, "b": 0}, inplace=True)
ttt_data["class"] = ttt_data["class"].astype(int)

print(ttt_data.head())
print(ttt_data.dtypes)

   TL  TM  TR  ML  MM  MR  BL  BM  BR  class
0   1   1   1   1  -1  -1   1  -1  -1      1
1   1   1   1   1  -1  -1  -1   1  -1      1
2   1   1   1   1  -1  -1  -1  -1   1      1
3   1   1   1   1  -1  -1  -1   0   0      1
4   1   1   1   1  -1  -1   0  -1   0      1
TL       int64
TM       int64
TR       int64
ML       int64
MM       int64
MR       int64
BL       int64
BM       int64
BR       int64
class    int64
dtype: object


<ipython-input-6-1c13d215ce16>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ttt_data.replace({'x': 1, 'o': -1, "b": 0}, inplace=True)


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [40]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

y = ttt_data["class"]
X = ttt_data.drop(columns=["class"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

model = Sequential()
model.add(Dense(6, activation="relu", input_shape=(9,)))
model.add(Dropout(.2))
model.add(Dense(2, activation="softmax"))

model.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=16, epochs=5)

score = model.evaluate(X_test, y_test)

print(f"Loss: {score[0]}, Accuracy: {score[1]}")

model.save("tic-tac-toe.keras")


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3870 - loss: 1.1090
Epoch 2/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4155 - loss: 0.9660
Epoch 3/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5368 - loss: 0.7667
Epoch 4/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5475 - loss: 0.7033
Epoch 5/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6248 - loss: 0.6525
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5611 - loss: 0.7005  
Loss: 0.6761614680290222, Accuracy: 0.5989583134651184


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [43]:
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
import numpy as np

model = load_model("tic-tac-toe.keras")

indices = [4, 7, 9, 12, 22]
X_sample = X_test.iloc[indices]
y_sample = y_test.iloc[indices]

pred_sample = model.predict(X_sample)

pred_classes = np.argmax(pred_sample, axis=1)

for i in range(len(indices)):
  print(f"Echte Klasse: {y_sample.iloc[i]}, Vorhergesagte Klasse: {pred_classes[i]}")

acc = accuracy_score(y_sample, pred_classes)

print("Accuracy Score:", acc)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Echte Klasse: 1, Vorhergesagte Klasse: 0
Echte Klasse: 1, Vorhergesagte Klasse: 1
Echte Klasse: 1, Vorhergesagte Klasse: 1
Echte Klasse: 1, Vorhergesagte Klasse: 0
Echte Klasse: 0, Vorhergesagte Klasse: 1
Accuracy Score: 0.4


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [54]:
new_model = Sequential()
new_model.add(Dense(32, activation="relu", input_shape=(9,)))
new_model.add(Dropout(.2))
new_model.add(Dense(16, activation="relu"))
new_model.add(Dropout(.2))
new_model.add(Dense(8, activation="relu"))
new_model.add(Dropout(.2))
new_model.add(Dense(2, activation="softmax"))

new_model.compile(optimizer=Adam(learning_rate=0.01), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

new_model.fit(X_train, y_train, batch_size=32, epochs=50)

score = new_model.evaluate(X_test, y_test)

print(f"Loss: {score[0]}, Accuracy: {score[1]}")

model.save("tic-tac-toe-improved.keras")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6398 - loss: 0.6584
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6647 - loss: 0.5582 
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7666 - loss: 0.5013 
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7801 - loss: 0.4700 
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8196 - loss: 0.4208 
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8812 - loss: 0.3161 
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9228 - loss: 0.2309 
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9211 - loss: 0.2294 
Epoch 9/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9475 - loss: 0.1526 
Epoch 10/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9598 - loss: 0.1414 
Epoch 11/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9730 - loss: 0.1269
Epoch 12/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.

**Which approach(es) did you find helpful to improve your model performance?**

> Blockzitat einfügen



In [ ]:
# Changing the epoch and the amount of neurons per layer.